Run this to get decision tree which we use as our classifier

In [34]:
%run "../DecisionTree/CS5350HW1.ipynb"

# Adaboost Algorithm

## Main Algorithm

In [35]:
#Define D_t
#for t=1,2,..,T
    #find h_t whose weighted classification error is better than chance
    #Compute a_t = 1/2ln((1-e_t)/e_t)
    #Update D_t+1(i) = D_t(i)/Z_t*e^{-a_ty_ih_t(x_i)}
#H_final = sgn(sum(a_th_t(x)))

def Adaboost(D_t, iterations, s_train, full_attributes, attributes, purity_function):
    classifiers = []
    alphas = []
    for i in range(iterations):
        print("Iteration:", i)
        root = createTree(s_train, attributes, full_attributes, purity_function, D_t, 1)
        error = findError(root, s_train, D_t, full_attributes)
        a_t = 0.5*np.log((1-error)/error)
        Z_t = np.sum(D_t)
        D_t_next = np.ones(len(s_train))
        for trainInd in range(len(s_train)):
            #travTree returns 1 if matched and 0 otherwise
            pred = travTree(root, s_train[trainInd], full_attributes)
            if(pred == 0):
                D_t_next[trainInd] = D_t[trainInd]/Z_t*np.e**(a_t)
            else:
                D_t_next[trainInd] = D_t[trainInd]/Z_t*np.e**(-a_t)
        print(D_t)
        print(D_t_next)
        print(error)
        displayTree(root)
        D_t = D_t_next
        classifiers.append(root)
        alphas.append(a_t)
    return classifiers, alphas
            
            

In [36]:
def findError(root, s_test, D_t, full_attributes):
  matched = 0
  for i in range(len(s_test)):
    #Note, (1-x) maps 0 -> 1 and 1 -> 0
    matched += D_t[i]*(1-travTree(root, s_test[i], full_attributes))
  return matched

## Finding accuracy

In [37]:
def findModelAccuracy(s_test, classifiers, alphas, full_attributes):
    matched = 0
    for test in s_test:
        h_f = computeHFinal(classifiers, alphas, test, full_attributes)
        print(h_f, test[len(test)-1])
        if(h_f > 0 and convertStringToBinary(test[len(test)-1]) > 0):
            matched += 1
        elif(h_f < 0 and convertStringToBinary(test[len(test)-1]) > 0):
             matched += 1
    return matched/len(s_test)


In [38]:
def computeHFinal(classifiers, alphas, x_i, full_attributes):
    final = 0
    for i in range(len(classifiers)):
        print(alphas[i])
        print(travTreeWithoutLabel(classifiers[i], x_i, full_attributes))
        final += alphas[i]*travTreeWithoutLabel(classifiers[i], x_i, full_attributes)
    return final

In [39]:
def convertStringToBinary(y_i):
    if(y_i == "yes"):
        return 1
    else:
        return -1

Pretty much travTree, but now will return 1 for a yes label and -1 for a no label

In [43]:
def travTreeWithoutLabel(root, datapoint, full_attr):
  #Stopping Condition, root no children
  if(len(root.children) == 0):
    if(root.label == "yes"):
      #print("matched")
      return 1
    else:
      #print("no match")
      return -1
  #Otherwise, traverse down
  curr_attr = root.label
  do_numeric = False
  if(full_attr[curr_attr] == "numeric"):
    do_numeric = True
  #print("current attribute: ", curr_attr)
  atr_ind = list(full_attr.keys()).index(curr_attr)
  for child in root.children:
    if(do_numeric):
      median = statistics.median(x[atr_ind] for x in root.s)
      #print("numeric: ", median, " ", datapoint[atr_ind])
      if((datapoint[atr_ind] < median and child.parentVal=="lower than median") or (datapoint[atr_ind] >= median and child.parentVal=="higher than median")):
        return travTreeWithoutLabel(child, datapoint, full_attr)
    else:
      if(child.parentVal == datapoint[atr_ind]):
        #print("Going down on val: ", child.parentVal)
        return travTreeWithoutLabel(child, datapoint, full_attr)

# Testing

In [44]:
iterations = 10
s_train = s_banktraining
s_test = s_banktesting
full_attributes = full_attributes_bank
attributes = remaining_attributes_bank
purity_function = entropy

D_1 = np.ones(len(s_train))/len(s_train)

classifiers, alphas = Adaboost(D_t=D_1, iterations=iterations, s_train=s_train, full_attributes=full_attributes, attributes=attributes, purity_function=purity_function)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print(alphas)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
findModelAccuracy(s_test=s_test, classifiers=classifiers, alphas=alphas, full_attributes=full_attributes)

Iteration: 0
[0.0002 0.0002 0.0002 ... 0.0002 0.0002 0.0002]
[7.35748644e-05 7.35748644e-05 5.43663931e-04 ... 7.35748644e-05
 7.35748644e-05 7.35748644e-05]
0.11920000000000136
~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ParentVal:  None
Label:  duration
~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ParentVal:  lower than median
Label:  no
ParentVal:  higher than median
Label:  no
----
Iteration: 1
[7.35748644e-05 7.35748644e-05 5.43663931e-04 ... 7.35748644e-05
 7.35748644e-05 7.35748644e-05]
[7.86041273e-05 7.86041273e-05 1.21171669e-03 ... 7.86041273e-05
 7.86041273e-05 7.86041273e-05]
0.32402370283668813
~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ParentVal:  None
Label:  duration
~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ParentVal:  lower than median
Label:  no
ParentVal:  higher than median
Label:  no
----
Iteration: 2
[7.86041273e-05 7.86041273e-05 1.21171669e-03 ... 7.86041273e-05
 7.86041273e-05 7.86041273e-05]
[0.00011862 0.00011862 0.00070346 ... 0.00011862 0.00011862 0.00011862]
0.722183144481344
~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Parent

KeyboardInterrupt: 